# Tests and examples of how to use the ts_auto_wrapper #

In [1]:
# python imports
from glob import glob
import ctypes
import struct
import sys
from pathlib import Path
# standard scientific packages
import numpy as np
import pandas
# repository specific modules
import vtu
PyVtu = vtu.PyVtu
import small_functions
import ts_auto_wrapper
TSWrapper = ts_auto_wrapper.TSWrapper

In [2]:
# import cffi # :( haven't managed to make it work

Test the wrapper: since the location of the trisurf library is not known, the module defines `TSWrapper` class which acts like a module. Both the module and the class have `help` documentation.  
*The path to trisurf need to be changed*

In [3]:
path_to_trisurf = Path("../cluster-trisurf")
if not path_to_trisurf.exists():
    path_to_trisurf = Path("../../cluster-trisurf")
    if not path_to_trisurf.exists():
        path_to_trisurf = Path("~/cluster_trisurf")
path_to_trisurf = path_to_trisurf.resolve()
ts=TSWrapper(path_to_trisurf)
help(ts)

Help on TSWrapper in module ts_auto_wrapper object:

class TSWrapper(builtins.object)
 |  TSWrapper(path_to_trisurf='/opt/workspace/msc_project/cluster-trisurf', more_types=None)
 |  
 |  Class that instantiate a trisurf wrapper from a path.
 |  
 |  Design to act like a module i.e.
 |  >>> from ts_auto_wrapper import TSWrapper
 |  >>> ts = TSWrapper('/path/to/trisurf/project/trisurf_ng')
 |  Exposes python binding to the library using CDLL.
 |  Everything is available in ts.X, but they are also organized by types with faster autocompletes:
 |      ts.ts_types: classes for types (ts_vertex, ts_tape, ts_vesicle, ...)
 |      ts.functions: functions (init_vertex, parseDump, ...)
 |      ts.globals: global variables
 |      ts.enums: enum definitions
 |      ts.TS_...: several #defines are hardcoded (e.g. TS_SUCCESS)
 |  and misc. things: ctype function POINTER, pointer; a pretty_print, and a byte_to_int function
 |  
 |  The raw functions with 'restype' and 'argtypes' are available as in

We can compare an example vtu file between a `PyVtu` object from `vtu.py` versus the wrappers `ctypes` results.  
*The .vtu file need to be changed. The file should match the trisurf version*

In [4]:
example_vtu_file = 'example/timestep_000009.vtu'
v = PyVtu(example_vtu_file)
vesicle = ts.parseDump(example_vtu_file) # pointer to the generated vesicle
vesicle

The wrapper for trisurf `parseDump` return a `ctype` class: this represent a pointer (`LP_to_`) to a struct (`ts_vesicle`).

We can test Samo's original version instead.  
*The path to trisurf nede to be changed*

In [5]:
# path_to_trisurf = '/opt/workspace/msc_project/trisurf_samo/trisurf-ng'
# ts = TSWrapper(path_to_trisurf)
# example_vtu_file = '/opt/workspace/msc_project/simulations/QA_tests/cluster_version/feature_tests/test_wrapper_with_old_version/timestep_000000.vtu'
# vesicle = ts.parseDump(example_vtu_file)
# v = PyVtu(example_vtu_file)

Trisurf heavily uses *pointers-to-structs* `vesicle->vlist->vtx[6]->x`. The ctype translation is with .contents: `vesicle.contents.vlist.contents[6].contents.x`.

In [6]:
print("type of vesicle center mass ",vesicle.contents.cm) # vesicle { ts_double cm[3] ...}
vesicle.contents.cm[:]

type of vesicle center mass  <ts_auto_wrapper.c_double_Array_3 object at 0x7f6b1987ec40>


[0.0, 0.0, 0.0]

We can use other functions in the library like `parsetape`.  
The CDLL functions have set `argtypes` and `restype`, and are wrapped with a few more processing:
* for strings, like paths to files, functions that need character arrays can recive strings.  
* for out parameters, functions that need pointer to doubles instantiate and return them.  
* If a string is modified by the function, the modified string is returned

The docstring of the function list the actual, pre-proccessing signature and the post-processing return type. The "Bare" c function is available in `ts._c_`*function*

In [7]:
ts.parsetape?

Signature:
ts.parsetape(
    *args,
    _base_c_function=<_FuncPtr object at 0x7f6b196d4dc0>,
    arg_types=[<class 'ctypes.LP_c_char'>],
    process_args=[<function TSWrapper.__init__.<locals>.process_str_to_buffer at 0x7f6b1984a290>],
    process_out=[<function TSWrapper.__init__.<locals>.process_neutral at 0x7f6b1984a0e0>, <function TSWrapper.__init__.<locals>.process_buffer_to_str at 0x7f6b19849f30>],
    check_args=[<function TSWrapper.__init__.<locals>.check_buffer_changed at 0x7f6b19849ea0>],
)
Docstring: Wrapper for parsetape with arguments , filename: c buffer or __str__. Return <class 'ts_auto_wrapper.LP_ts_tape'>, filename: c buffer or str
File:      ~/yoav/trisurf-python/ts_auto_wrapper.py
Type:      function

We can use trisurf to parse the default tape in the `/src/` folder  
This works because `ts.parsetape` knows to first convert the input object to string with `str()` (in this case, a `Path` )

In [8]:
base_tape=ts.parsetape(ts.path_to_trisurf/'src/tape')
base_tape

We can look at the tape using this helper function: this is meant to more easily print the tape or other ctypes structures

In [9]:
print(ts.pretty_print_struct(base_tape))

    tape_text <ctypes.LP_c_char object at 0x7f6b1987f640>
    R_nucleus 0.0
    R_nucleusX 0.0
    R_nucleusY 0.0
    R_nucleusZ 0.0
    xkA0 1.0
    xkV0 1.0
    V0 0.0
    A0 0.0
    Vfraction 1.0
    constvolprecision 1e-14
    xk0 20.0
    xk2 -20.0
    dmax 1.7
    dmin_interspecies 1.2
    stepsize 0.15
    kspring 800.0
    xi 100.0
    pressure 0.0
    c0 0.5
    d0 0.5
    w 1.0
    F 1.0
    plane_d 10.0
    plane_F 1.0
    vicsek_strength 0.1
    vicsek_radius 4.0
    adhesion_z -5.0
    adhesion_cutoff 1.0
    adhesion_strength 1.0
    adhesion_radius 5.0
    adhesion_scale 5.0
    adhesion_factor 2.0
    max_dihedral_angle_cosine -0.1
    mcsweeps 1000
    random_seed 0
    iterations 10
    inititer 0
    nshell 10
    ncxmax 100
    ncymax 100
    nczmax 100
    number_of_vertices_with_c0 50
    npoly 0
    nmono 20
    internal_poly 0
    nfil 0
    nfono 3
    shc 0
    pressure_switch b'\x00'
    volume_switch b'\x00'
    area_switch b'\x00'
    quiet b'\x00'
    plan

Let's compare the wrapper tape with the `small_functions.py` tape options extraction.

In [10]:
tstape = ts.pretty_print_struct(base_tape)
with open(ts.path_to_trisurf/'src/tape','r') as f:
    pytape = small_functions.get_tape_options(f.read())
all_lines = {}
for line1 in tstape.splitlines():
    first=line1.split()[0]
    if first in pytape:
        all_lines[first] = [line1, pytape[first]]
    else:
        all_lines[first] = [line1, None]
pandas.DataFrame(all_lines, index=['c','python'])

,<ts_auto_wrapper.LP_ts_tape,tape_text,R_nucleus,R_nucleusX,R_nucleusY,R_nucleusZ,xkA0,xkV0,V0,A0,...,plane_confinement_switch,allow_center_mass_movement,force_balance_along_z_axis,prevent_obtuse_triangles,debug_fields,adhesion_geometry,adhesion_model,bond_model,curvature_model,force_model
c,<ts_auto_wrapper.LP_ts_tape object at 0x7f6b19...,tape_text <ctypes.LP_c_char object at 0x7f...,R_nucleus 0.0,R_nucleusX 0.0,R_nucleusY 0.0,R_nucleusZ 0.0,xkA0 1.0,xkV0 1.0,V0 0.0,A0 0.0,...,plane_confinement_switch b'\x00',allow_center_mass_movement b'\x00',force_balance_along_z_axis b'\x00',prevent_obtuse_triangles b'\x00',debug_fields b'\x01',adhesion_geometry b'\x00',adhesion_model b'\x00',bond_model b'\x00',curvature_model b'%',force_model b'\x00'
python,None,None,0,0,0,0,1.0,1.0,0,0,...,0,0,0,None,1,0,0,0,37,0


Both agree on their values of the options (this is not trivial, since structure misalignment can corrupt the values)

Here we look at the spontaneous curvature of each vertex `vtx.c` with id between 50 to 100. This is not exactly ergonomic.

In [11]:
np.array([vesicle.contents.vlist.contents.vtx[i].contents.c for i in range(50,100)])

array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. ])

A helper function `iter_xlist` exists in the wrapper. It defaults to iterating over vlist structs

In [12]:
np.array([x.c for x in ts.iter_xlist(vesicle)])[50:100]

array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. ])

But it can be used to iterate over the other `x`lists, and to produce the pointers instead

In [13]:
t=next(ts.iter_xlist(vesicle,'tlist',as_pointer=True))
t.contents.xnorm, t.contents.ynorm,t.contents.znorm

(0.15662035189796142, -0.28231929755143637, -0.9464490898096011)

In [14]:
cells_dict = {}
for cell in ts.iter_xlist(vesicle,'clist'):
    if cell.nvertex!=0:
        cells_dict[cell.idx]=cell.nvertex
print(f"cells occupied: {len(cells_dict)}/{vesicle.contents.clist.contents.cellno}, for occupied, mean occupation {np.array((*cells_dict.values(),)).mean()}+-{np.array((*cells_dict.values(),)).std()}")

cells occupied: 502/1000000, for occupied, mean occupation 1.0+-0.0


We can compare the mean curvature from trisurf versus the mean curvature from the .vtu. There may be factors of 2.

In [15]:
ts.functions.vesicle_meancurvature(vesicle), v.mean_curvature.sum()

(39.11237755627411, 78.22475511254822)

In [16]:
ts.functions._c_direct_force_energy.argtypes

[ts_auto_wrapper.LP_ts_vesicle,
 ts_auto_wrapper.LP_ts_vertex,
 ts_auto_wrapper.LP_ts_vertex]

Here we look at the force acting on a vertex directly based on the CDLL:  
* we take the first active vertex `vtx`
* we create three dummy vertices with position at 1 unit before
* we calculate the work on each vertex with using the dummy vertex as the starting position
* $W=-f\cdot dx$, so we get the force in each direction
* we then compare it with the PyVtu

In [17]:
first_active = v.indices[v.type==47][0]

vtx=vesicle.contents.vlist.contents.vtx[first_active]
print(f"z in first active vertex:\nvlist->vtx[{first_active}]->z={vtx.contents.z}",f"\nPyVtu z:",v.pos[first_active,2])

z in first active vertex:
vlist->vtx[0]->z=5.916821879177588 
PyVtu z: 5.916821879177588


In [18]:
# dummy starting positions: dx=x̂, dx=ŷ dx=ẑ
vtx_oldx=ts.ts_vertex(x=vtx.contents.x-1, y=vtx.contents.y,   z=vtx.contents.z)
vtx_oldy=ts.ts_vertex(x=vtx.contents.x,   y=vtx.contents.y-1, z=vtx.contents.z)
vtx_oldz=ts.ts_vertex(x=vtx.contents.x,   y=vtx.contents.y,   z=vtx.contents.z-1)

In [19]:
fx = -ts.direct_force_energy(vesicle,vtx,ctypes.pointer(vtx_oldx)) # using the wrapper function

fy = -ts.functions._c_direct_force_energy(vesicle,vtx,ctypes.pointer(vtx_oldy)) # using the underlying cdell function with restype and argtype

ts.cdll.direct_force_energy.restype=ctypes.c_double # using the cdll function directly
fz = -ts.cdll.direct_force_energy(vesicle, vtx, ctypes.pointer(vtx_oldz))

print("trisurf force:",fx,fy,fz,"vtu force: ",v.force[first_active])

trisurf force: -0.15401958646749273 0.21741111965676563 0.9638518413293475 vtu force:  [-0.14960309  0.20967829  0.96625769]


Some more uses:

Parsing types based on the known types

In [20]:
ts_auto_wrapper.parse_type("int **stuff",ts.ts_types)

('int', {'stuff': ts_auto_wrapper.LP_LP_c_int})

byte to int

In [21]:
bond_model = vesicle.contents.tape.contents.bond_model
bond_model, ts.byte_to_int(bond_model)

(b'\x00', 0)

Clusterize vesicle from python  
We need to create a cluster list, set all the cluster pointers in the vertices to NULL, and then run the clusterization algorithm

In [22]:
cluster_list = ts.init_cluster_list()
for vtxPtr in ts.iter_xlist(vesicle,as_pointer=True):
    vtxPtr.contents.cluster=None
ts.clusterize_vesicle(vesicle,cluster_list)

True

In [23]:
cluster_list.contents.n

32

We can manually run a timestep

In [24]:
ts.functions.single_timestep(vesicle)

(True, 0.45219123505976094, 0.022576361221779546)

#### Test the --tape-options method
This will only work in cluster-trisurf versions!

Ctypes is a little awkward for strings: in principle, we need to use string buffer if our functions change anything

In [25]:
tape="#TAPE\nnshell=50\niterations=100\nopt=7\n#DONE"
opts=",nshell=40,,-iterations,opt9=blue,-opt=4,#hello,,,"
print(tape,'\nreplacement options:',opts)

#TAPE
nshell=50
iterations=100
opt=7
#DONE 
replacement options: ,nshell=40,,-iterations,opt9=blue,-opt=4,#hello,,,


In [26]:
my_tape=ctypes.create_string_buffer(tape.encode("ascii"), size=1024)
my_opts=ctypes.create_string_buffer(opts.encode("ascii"),size=1024)

In [27]:
my_tape

Directly use the CDLL function without pre-processing

In [28]:
ts.functions._c_update_tapetxt(my_tape,my_opts)

b'\x00'

In [29]:
print(my_tape.value.decode())

#TAPE
nshell=40

opt=7
#DONE

#--tape-options
#removed iterations
opt9=blue
#hello



The pre-process wrapper functions will compromise: it takes regular strings, but if they change, it returns the changed string

In [30]:
ret=ts.update_tapetxt(tape,opts)

In [31]:
ret

(True,
 '#TAPE\nnshell=40\n\nopt=7\n#DONE\n\n#--tape-options\n#removed iterations\nopt9=blue\n#hello\n')

In [32]:
print(ret[1])

#TAPE
nshell=40

opt=7
#DONE

#--tape-options
#removed iterations
opt9=blue
#hello



So if nothing is changed, the wrapper does not return the string

In [33]:
ret=ts.update_tapetxt(tape,"")

In [34]:
ret

True